In [83]:
import numpy as np
import cv2
import os
import cvlib as cv
from keras.preprocessing.image import img_to_array
from keras.models import load_model

In [84]:
def getClassName(classIndex):
    "A basic function to get the class name."
    if classIndex==0:
        return "Sad"
    
    elif classIndex==1:
        return "Shocked"
    
    elif classIndex==2:
        return "Happy"

In [85]:
model = load_model("final_model.h5") # Loading the pretrained model to predict

***

In [137]:
facess = cv2.imread("sad.jpg")
facess = cv.detect_face(facess)

In [141]:
facess[0][3]

[270, 543, 427, 795]

In [ ]:
# başka bir face_detection mekanizması bul.

***

In [142]:
cap = cv2.VideoCapture(0)
rect_color = (0,255,0)
# Above^ Class instance for video capturing.
# If a video name is entered, captures the specified video.

threshold = 0.80 * 100

while cap.isOpened(): # Repeats until the key "q" is pressed.
    
    ret, frame = cap.read() # Starts capturing. 
    # ret is True or False to represent if capturing is successful.
    # frame is the image to process.
    if not ret:
        print("Can't receive the frame.")
        break
        
    # FACE RECOGNITION    
    # Note: input image (numpy array, BGR order, dtype - uint8)
    faces, confidences = cv.detect_face(frame)
    
    
    for _, f in enumerate(faces):
        # corner points: 
        (X0, Y0) = (f[0], f[1])    # 0: bottom left points
        (X1, Y1) = (f[2], f[3])    # 1: top right points
        
        cv2.rectangle(img=frame, 
                      pt1=(X0, Y0), 
                      pt2=(X1, Y1), 
                      color=rect_color, 
                      thickness=2)
        cropped_frame = np.copy(frame[Y0:Y1, X0:X1])

        if (cropped_frame.shape[0]) < 10 or (cropped_frame.shape[1]) < 10:
            continue
    
        # preprocessing on the cropped frame
        cropped_frame = cv2.resize(cropped_frame, (64,64))
        cropped_frame = cropped_frame.astype("float64") / 255.0
        cropped_frame = np.expand_dims(cropped_frame, axis=0)
        confidences = model.predict(cropped_frame)[0]    # Probability values for all classes
        max_probability = max(confidences)*100
        classIndex = np.argmax(confidences)
        className = getClassName(classIndex)



        if max_prob > threshold:
            frame_text = f"Sentiment: {className} ({max_probability:.2f}%)"
            if className == "Shocked":
                rect_color = text_color = (0, 255, 255)
            elif className == "Sad":
                rect_color = text_color = (255, 255, 0)
            else:
                rect_color = text_color = (0, 255, 0)
        else:
            frame_text = "Reading..."

        Y0 = (Y0 - 10) if (Y0 - 10) > 10 else (Y0 + 10)
        cv2.putText(img=frame, 
                    text=frame_text, 
                    org=(X0, Y0), 
                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.6, 
                    color=text_color, 
                    thickness=2)  
    
    cv2.imshow("Face_Sentiment", frame) # Showing the video. Window's name is "Face_Sentiment"
    
    if cv2.waitKey(1) > 0:    # Repeats until the key "q" is pressed.
        break

cap.release() # Releases the camera after a key is pressed
cv2.destroyWindow("Face_Sentiment") # Closes the window named "Face_Sentiment"
# cv2.waitKey() # Necessary for not having trouble with the closing window

In [58]:
print(model.predict(img_shock_numpy))
print(getClassName(np.argmax(model.predict(img_shock_numpy), axis=-1)))

[[3.5308785e-05 9.9714428e-01 2.8203626e-03]]
Shocked


In [59]:
print(model.predict(img_happy_numpy))
print(getClassName(np.argmax(model.predict(img_happy_numpy), axis=-1)))

[[2.7099004e-04 3.6971833e-04 9.9935931e-01]]
Happy


In [60]:
# false img. a rose picture
print(model.predict(img_false_numpy))
print(getClassName(np.argmax(model.predict(img_false_numpy), axis=-1)))

[[5.6500215e-14 3.4709935e-04 9.9965286e-01]]
Happy


***
***

In [29]:
cap.release()

In [ ]:
frame.shape

In [ ]:
yy.shape